In [ ]:
import igl
import numpy as np
import meshplot as mp
import torch
from math import pi
# import matplotlib.pyplot as plt

import pythreejs as p3s
import ipywidgets as iw
from IPython.display import clear_output

EPS  = 0.00000000001
EPS1 = 0.0000001
EPS2 = 0.00001

: 

# Load meshes and display

In [3]:
with np.load('data/octa_sphere_5.npz') as npl:
    v_s, f_s = npl['v'], npl['f']

#Mesh
# v_m, f_m = igl.read_triangle_mesh("data/deer.obj")
v_m, f_m = igl.read_triangle_mesh("data/cat.obj")

#Cage
# v_c, f_c = igl.read_triangle_mesh("data/deer_cage.obj")
# v_c, f_c = igl.read_triangle_mesh("data/cat_cage.obj")
v_c, f_c = igl.read_triangle_mesh("data/cc_part2.obj")

n = igl.per_face_normals( v_c, f_c, np.array([1.0, 0, 0]) ) #normalized

plt = mp.plot(v_m, f_m)

lines_start = np.vstack( [v_c[f_c[:,0]], v_c[f_c[:,1]], v_c[f_c[:,2]]] )
lines_end =  np.vstack( [v_c[f_c[:,1]], v_c[f_c[:,2]], v_c[f_c[:,0]]] )

plt.add_lines( lines_start, lines_end, shading={"line_color": "red"})
plt.add_points(v_c, shading={"point_color": "green", "point_size": 0.2})

cof = (v_c[f_c[:,0]]+v_c[f_c[:,1]]+v_c[f_c[:,2]])/3 # cage face의 center of mass

v_mci = np.array([0, 0, 0]) # shape vertex중 cage 안에 있는 vertex 들
v_mco = np.array([0, 0, 0]) # shape vertex중 cage 밖에 있는 vertex 들
v_mciidx = np.array([], dtype = np.int16) # cage 안에 있는 vertex 들의 index

for i in range(v_m.shape[0]):
    out = 0
    vtocof = cof-v_m[i]
    for j in range(f_c.shape[0]):
        if(vtocof[j]@n[j]<-0.001):
            out = 1
            break
    if(out == 0):
        v_mci=np.vstack((v_mci, v_m[i]))
        v_mciidx = np.append(v_mciidx, np.array([i]))
    else:
         v_mco=np.vstack((v_mco, v_m[i]))
        
v_mci = v_mci[1:]
v_mco = v_mco[1:]

# f_mci = np.array([0, 0, 0]) # shape face중 cage 안에 있는 face 들
# f_mco = np.array([0, 0, 0]) # shape face중 cage 밖에 있는 face 들
# for i in range(f_m.shape[0]):
#     if((f_m[i, 0] in v_mciidx) and (f_m[i, 1] in v_mciidx) and (f_m[i, 2] in v_mciidx)):
#         f_mci=np.vstack((f_mci, f_m[i]))
#     else:
#         f_mco=np.vstack((f_mco, f_m[i]))

# f_mci = f_mci[1:]
# f_mco = f_mco[1:]
# for idx, i in enumerate(v_mciidx):
#     f_mci[np.where(f_mci==i)] = idx
# ui = mp.plot(v_mci, f_mci)
# ui.add_mesh(v_m, f_mco, c=np.array([1,0,0]))

# For partial cage deformation
St = v_m.copy() # 그냥 = 으로 복사하면 같이 바뀜
St[v_mciidx] = np.array([0, 0, 0])

plt.add_points(v_mci, shading={"point_color": "blue", "point_size": 0.2})
print()


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.317513…

# Green coordinates

In [ ]:
# Partial Deform을 위해 추가적으로 필요한 data
# 1. cage 안의 vertex 와 그 vertex의 index
# 2. cage 밖의 vertex 고정을 위한 matrix St

In [4]:
# deformations functions
# By tensor

def GCTriInt(p, v1, v2, eta):
    
    alpha = torch.acos((v2-v1)@(p-v1)/(torch.norm(v2-v1)*torch.norm(p-v1)))
    beta = torch.acos((v1-p)@(v2-p)/(torch.norm(v1-p)*torch.norm(v2-p)))
    
    #alpha(or beta) 가 거의 pi나 0에 가까울 경우 삼각형의 넓이가 거의 0 이라 적분값이 0 
    # if abs(alpha - np.pi) < EPS or abs(alpha) < EPS: return 0 
    # if abs(beta - np.pi) < EPS or abs(beta) < EPS: return 0
    
    l = (torch.norm(p-v1)**2)*(torch.sin(alpha)**2)
    c = torch.norm(p-eta)**2
    
   
    
    theta = pi - alpha
    S = torch.sin(theta)
    C = torch.cos(theta)
    
    d1 = torch.sqrt(l + c*(S**2))
    d2 = c*(1+C)+l+torch.sqrt(l**2+l*c*(S**2))
    
    I1 = -(1/2)*torch.sign(S)*(2*torch.sqrt(c)*torch.atan(torch.sqrt(c)*C/d1))\
                               + torch.sqrt(l)*torch.log(2*torch.sqrt(l)*(S**2)*(1-2*c*C/d2)/((1-C)**2))
    
    theta = pi - alpha - beta
    S = torch.sin(theta)
    C = torch.cos(theta)
    
    d1 = torch.sqrt(l + c*(S**2))
    d2 = c*(1+C)+l+torch.sqrt(l**2+l*c*(S**2))
    
    I2 = -(1/2)*torch.sign(S)*(2*torch.sqrt(c)*torch.atan(torch.sqrt(c)*C/d1))\
                               + torch.sqrt(l)*torch.log(2*torch.sqrt(l)*(S**2)*(1-2*c*C/d2)/((1-C)**2))

    return -(1/(4*pi))*abs(I1-I2-torch.sqrt(c)*beta)




def genCoords(V, T, v_m, vidx, t_len):
    # Input (tensor)
    # V : (cv, 3) (cv: # of cage vertex) - cage vertices (sorted by ID)
    # T : (cf, 3) (cf: # of cage face) - vertices IDs of faces
    # v_m : (n, 3) (n: # of shape vertex) - vertices of shape
    
    # Output
    # phi : (n, cv) : coordinate about cage's vertex 
    # psi : (n, cf) : coordinate about cage's face's normal
    
    # For a chunk cage deformation
    # phi = torch.zeros(len(v_m), len(V))
    # psi = torch.zeros(len(v_m), len(T))

    # For partial deformation
    phi = torch.zeros(t_len, len(V))
    psi = torch.zeros(t_len, len(T))
    for eta_idx, eta in enumerate(v_m):
        clear_output(wait=True)
        print(eta_idx+1,"/",len(v_m)," ", (eta_idx+1)/len(v_m)*100,"%")
        for f_idx, f in enumerate(T):
            v = torch.tensor([V[f[0]], V[f[1]], V[f[2]]], dtype = float)
            # print("v")
            # print(v)
            # print("eta")
            # print(eta)
            for l in range(3):
                v[l] -= eta 
            p = (v[0]@n[f_idx])*n[f_idx]
            
            s = torch.zeros(3, dtype = float)
            I = torch.zeros(3, dtype = float)
            II = torch.zeros(3, dtype = float)
            N = torch.zeros((3, 3), dtype = float)
            
            for l in range(3):
                s[l] = torch.sign(torch.cross(v[l%3]-p, v[(l+1)%3]-p)@n[f_idx])          
                I[l] = GCTriInt(p, v[l%3], v[(l+1)%3], torch.zeros(3))              
                II[l] = GCTriInt(torch.zeros(3),  v[(l+1)%3], v[l%3], torch.zeros(3))
                q = torch.cross(v[(l+1)%3],  v[l%3])
                N[l] = q/torch.norm(q)
            
            I_f = - abs(s[0]*I[0]+s[1]*I[1]+s[2]*I[2])
            # For a chunk cage deformation
            # psi[eta_idx, f_idx] = -I_f
            
            # For partial deformation
            psi[vidx[eta_idx], f_idx] = -I_f
            w = I_f*n[f_idx] + II[0]*N[0]+II[1]*N[1]+II[2]*N[2]
            
            if(torch.norm(w) > 0.00001):
                for l in range(3):
                    # For a chunk cage deformation
                    # phi[eta_idx, f[l]] += (N[(l+1)%3]@w)/(N[(l+1)%3]@v[l])
                    
                    # For partial deformation
                    phi[vidx[eta_idx], f[l]] += (N[(l+1)%3]@w)/(N[(l+1)%3]@v[l]) #하나의 face의 한 꼭짓점에 대한 phi component 계산값을 하나씩 축적해줌                 
    
    return psi.T, phi.T

psi, phi = genCoords(v_c, f_c, v_mci, v_mciidx, len(v_m)) #psi : for face's normal, phi: for vertex
psi = psi.numpy()
phi = phi.numpy()



115 / 115   100.0 %


**Some modification to the meshplot library are necessary since otherwise the camera is reset every time**

In [13]:
#Meshplot modifications
def gen_circle(width=256, height=256):
    xx, yy = np.mgrid[:width, :height]
    circle = (xx - width/2 + 0.5) ** 2 + (yy - height/2 + 0.5) ** 2
    array = np.ones((width, height, 4), dtype='float32')
    array[:, :, 0] = (circle <= width)
    array[:, :, 1] = (circle <= width)
    array[:, :, 2] = (circle <= width)
    array[:, :, 3] = circle <= width
    return array

def remove_object(viewer, obj_id):
    if obj_id not in viewer._Viewer__objects:
        print("Invalid object id. Valid ids are: ", list(viewer._Viewer__objects.keys()))
        return
    viewer._scene.remove(viewer._Viewer__objects[obj_id]["mesh"])
    del viewer._Viewer__objects[obj_id]
    
def add_object(viewer, obj, parent=None):
    if not parent: # Object is added to global scene and objects dict
        viewer._Viewer__objects[viewer._Viewer__cnt] = obj
        viewer._Viewer__cnt += 1
        viewer._scene.add(obj["mesh"])
    else: # Object is added to parent object and NOT to objects dict
        parent.add(obj["mesh"])

    return viewer._Viewer__cnt - 1

def add_line_geometry(viewer, lines, shading, obj=None):
    lines = lines.astype("float32", copy=False)
    mi = np.min(lines, axis=0)
    ma = np.max(lines, axis=0)

    geometry = p3s.LineSegmentsGeometry(positions=lines.reshape((-1, 2, 3)))
    material = p3s.LineMaterial(linewidth=shading["line_width"], color=shading["line_color"])
                #, vertexColors='VertexColors'),
    lines = p3s.LineSegments2(geometry=geometry, material=material) #type='LinePieces')
    line_obj = {"geometry": geometry, "mesh": lines, "material": material,
                "max": ma, "min": mi, "type": "Lines", "wireframe": None}

    if obj:
        return add_object(viewer, line_obj, obj), line_obj
    else:
        return add_object(viewer, line_obj)


def add_points(viewer, points, c=None, shading={}, obj=None, **kwargs):
    shading.update(kwargs)
    if len(points.shape) == 1:
        if len(points) == 2:
            points = np.array([[points[0], points[1], 0]])
    else:
        if points.shape[1] == 2:
            points = np.append(
                points, np.zeros([points.shape[0], 1]), 1)
    sh = viewer._Viewer__get_shading(shading)
    points = points.astype("float32", copy=False)
    try:
        mi = np.min(points, axis=0)
    except ValueError: 
        mi = 0
    try:
        ma = np.max(points, axis=0)
    except ValueError:  
        ma = 0
       

    # mi = np.min(points, axis=0)
    # ma = np.max(points, axis=0)
    

    g_attributes = {"position": p3s.BufferAttribute(points, normalized=False)}
    m_attributes = {"size": sh["point_size"]}

    if sh["point_shape"] == "circle": # Plot circles
        tex = p3s.DataTexture(data=gen_circle(16, 16), format="RGBAFormat", type="FloatType")
        m_attributes["map"] = tex
        m_attributes["alphaTest"] = 0.5
        m_attributes["transparency"] = True
    else: # Plot squares
        pass

    colors, v_colors = viewer._Viewer__get_point_colors(points, c, sh)
    if v_colors: # Colors per point
        m_attributes["vertexColors"] = 'VertexColors'
        g_attributes["color"] = p3s.BufferAttribute(colors, normalized=False)

    else: # Colors for all points
        m_attributes["color"] = colors

    material = p3s.PointsMaterial(**m_attributes)
    geometry = p3s.BufferGeometry(attributes=g_attributes)
    points = p3s.Points(geometry=geometry, material=material)
    point_obj = {"geometry": geometry, "mesh": points, "material": material,
        "max": ma, "min": mi, "type": "Points", "wireframe": None}

    if obj:
        return add_object(viewer, point_obj, obj), point_obj
    else:
        return add_object(viewer, point_obj)
    
def add_lines(viewer, beginning, ending, shading={}, obj=None, **kwargs):
    shading.update(kwargs)
    if len(beginning.shape) == 1:
        if len(beginning) == 2:
            beginning = np.array([[beginning[0], beginning[1], 0]])
    else:
        if beginning.shape[1] == 2:
            beginning = np.append(
                beginning, np.zeros([beginning.shape[0], 1]), 1)
    if len(ending.shape) == 1:
        if len(ending) == 2:
            ending = np.array([[ending[0], ending[1], 0]])
    else:
        if ending.shape[1] == 2:
            ending = np.append(
                ending, np.zeros([ending.shape[0], 1]), 1)

    sh = viewer._Viewer__get_shading(shading)
    lines = np.hstack([beginning, ending])
    lines = lines.reshape((-1, 3))
    return add_line_geometry(viewer, lines, sh, obj)

# User interface

In [17]:
selected_is_empty = True 
edge_obj = 2
green_points_obj = 3
red_points_obj = -1 #Not created yet
x_range, y_range, z_range  = np.max(v_c, axis=0) - np.min(v_c, axis=0) +5

grid = iw.GridspecLayout(4, 2)

def drawPoints():
    #global selected_is_empty
    global selected_is_empty, green_points_obj, edge_obj, red_points_obj
    
    remove_object(ui, green_points_obj)
    if not selected_is_empty: remove_object(ui, red_points_obj)
    not_selected = np.where(current_selected == 0)
    selected_points = np.where(current_selected == 1)
    #Add not selected points
    add_points(ui, v_cc[not_selected], shading={"point_color": "green", "point_size": 0.1})
    green_points_obj = np.max([edge_obj, green_points_obj, red_points_obj]) + 1
    #Add selected points
    if np.size(selected_points) > 0:
        add_points(ui, v_cc[selected_points], shading={"point_color": "blue", "point_size": 0.2})
        selected_is_empty = False
        red_points_obj = np.max([edge_obj, green_points_obj, red_points_obj]) + 1
    else: selected_is_empty = True

    
def displacePoints():
    #global selected_is_empty
    global selected_is_empty, green_points_obj, edge_obj, red_points_obj
    
    #remove current cage
    remove_object(ui, edge_obj)
    if not selected_is_empty: remove_object(ui, red_points_obj)
    not_selected = np.where(current_selected == 0)
    selected_points = np.where(current_selected == 1)
    #Add selected points
    if np.size(selected_points) > 0:
        add_points(ui, v_cc[selected_points], shading={"point_color": "blue", "point_size": 0.2})
        selected_is_empty = False
        red_points_obj = np.max([edge_obj, green_points_obj, red_points_obj]) + 1
    else: selected_is_empty = True
    #Add edges
    lines_start = np.vstack( [v_cc[f_c[:,0]], v_cc[f_c[:,1]], v_cc[f_c[:,2]]] )
    lines_end =  np.vstack( [v_cc[f_c[:,1]], v_cc[f_c[:,2]], v_cc[f_c[:,0]]] )
    add_lines(ui, lines_start, lines_end, shading={"line_color": "red"})
    edge_obj = np.max([edge_obj, green_points_obj, red_points_obj]) + 1 
            
#interface
v_ref = np.copy(v_c)
v_cc = np.copy(v_c) #current cage

current_selected = np.zeros(len(v_c), dtype=int)
displ = np.zeros([len(v_c),3])

select_button   = iw.Button(description="Select")
deselect_button = iw.Button(description="Deselect")
deselect_all_button = iw.Button(description="Deselect all")
clear_button = iw.Button(description="Clear displace sliders")

# Set Callback
def select_clicked(b):
    new_points = np.where(np.linalg.norm(v_cc - sf.coord[1:],axis=1) < sf.coord[0])[0]
    current_selected[new_points] = 1
    drawPoints()
    
def deselect_clicked(b):
    new_points = np.where(np.linalg.norm(v_cc - sf.coord[1:],axis=1) < sf.coord[0])[0]
    current_selected[new_points] = 0
    drawPoints()
    
def deselect_all_clicked(b):
    current_selected[:] = 0
    drawPoints()
    
def clear_clicked(b):
    global current_selected
    tmp = np.copy(current_selected)
    current_selected[:] = 0
    grid[1,1].value = 0
    grid[2,1].value = 0
    grid[3,1].value = 0
    current_selected = tmp
    drawPoints()
    v_ref[:] = v_cc[:] 
    displ[:] = 0
    
select_button.on_click(select_clicked)
deselect_button.on_click(deselect_clicked)
deselect_all_button.on_click(deselect_all_clicked)
clear_button.on_click(clear_clicked)


# Meshplot
ui = mp.plot(v_m, f_m)
ui.add_mesh(v_s*0.1, f_s, c=np.array([1,0,0]))
lines_start = np.vstack( [v_cc[f_c[:,0]], v_cc[f_c[:,1]], v_cc[f_c[:,2]]] )
lines_end =  np.vstack( [v_cc[f_c[:,1]], v_cc[f_c[:,2]], v_cc[f_c[:,0]]] )
ui.add_lines(lines_start, lines_end, shading={"line_color": "red"})
ui.add_points(v_cc, shading={"point_color": "green", "point_size": 0.1})

# Display Buttons
display(iw.HBox([select_button, deselect_button, deselect_all_button, clear_button]))

#Selection ball
def sf(radius,x,y,z):
    ui.update_object(oid = 1, vertices = v_s*radius + np.array([x,y,z]))
    #ui.remove_object(10)
    sf.coord = [radius,x,y,z]
    

grid[0,0] = iw.FloatSlider(min=0.1, max=1, value=0.1, description='Radius')
grid[1,0] = iw.FloatSlider(min=-2.5, max=2.5, value=0, description='x')
grid[2,0] = iw.FloatSlider(min=-2.5, max=2.5, value=0, description='y')
grid[3,0] = iw.FloatSlider(min=-2.5, max=2.5, value=0, description='z')

iw.interactive_output(sf, {
            'radius': grid[0,0],
            'x': grid[1,0],
            'y': grid[2,0],
            'z': grid[3,0]
})

#Displacements

grid[1,1] = iw.FloatSlider(min=-x_range/3, max=x_range/3, value=0, description='displace_x')
grid[2,1] = iw.FloatSlider(min=-y_range/3, max=y_range/3, description='displace_y')
grid[3,1] = iw.FloatSlider(min=-z_range/3, max=z_range/3, description='displace_z')

def displace(displace_x,displace_y,displace_z):
    global v_cc
    current_displ = np.array([displace_x,displace_y,displace_z])
    displ[current_selected == 1] = current_displ
    v_cc = v_ref + displ
    
    if not selected_is_empty: 
        displacePoints()
        #update v 
        # Tensor랑 numpy랑 계산될때 deform이 잘 안됐음
        # sign이 이론적으로 계산된거데로 하면 object가 거꾸로 바뀜 -를 붙여줘야함(위에 gc 계산할때도 비슷한 문제 있었음)
        
        # For a chunk cage deformation
        #  deformed = - (phi.T@v_cc + psi.T@n)
        
        # For partial deformation
        deformed = - (phi.T@v_cc + psi.T@n - St)
        ui.update_object(oid = 0, vertices = deformed) #지금 분리되는 모형을 따라가려면 
    return




iw.interactive_output(displace, {
            'displace_x': grid[1,1],
            'displace_y': grid[2,1],
            'displace_z': grid[3,1]
})

grid

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.317513…

GridspecLayout(children=(FloatSlider(value=0.1, description='Radius', layout=Layout(grid_area='widget001'), ma…